# Stock price project

Require libs necessary to this project

In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import os

Yahoo site with all information 

In [5]:
url = 'https://br.financas.yahoo.com/noticias/acoes-mais-negociadas?offset=0&count=100'
response = urlopen(url)
html = response.read()

### Caso de erro na leitura

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

#### vai até network, index.php, request headers, user-agent

url = 'https://br.financas.yahoo.com/noticias/acoes-mais-negociadas'
headers = {}

try: 
    req = Request(url, headers = headers)
    response = urlopen(req)

except HTTPError as e:
    print(e.status, e.reason)
  
except URLError as e:
    print( e.reason)

In [6]:
html = html.decode('utf-8')

In [7]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

html = trata_html(html)
soup = BeautifulSoup(html, features="html.parser")

In [8]:
alldata = soup.find_all('tbody')
stock_data = []
stock_history = []
new_ya = 'https://br.financas.yahoo.com'

for stocks in alldata:
    stock = stocks.find_all('tr')
    for infos in stock:
        info = infos.find_all('td')
        a = {'stock': info[0].get_text(), 'company': info[1].get_text(), 'price': info[2].get_text(),
         'change': info[3].get_text(), 'perc_change': info[4].get_text(), 'volume': info[5].get_text(),
         'vol_mean_3m' : info[6].get_text(), 'market_cap': info[7].get_text(), 'PL_index': info[8].get_text()}
        stock_data.append(a)
        site_aux = info[0].find('a')
        site_href = site_aux.get('href')
        new_yan = new_ya + site_href.split('?')[0] + '/history?period1'
        stock_history.append(new_yan)


Table with 200 stock informations

In [9]:
pd.DataFrame(stock_data).head(10)

,stock,company,price,change,perc_change,volume,vol_mean_3m,market_cap,PL_index
0,PETR4.SA,Petróleo Brasileiro S.A. - Petrobras,"24,00","+0,76","+3,27%",0,"91,749M","309,494B","44,04"
1,BBDC4.SA,Banco Bradesco S.A.,"26,74","+0,27","+1,02%",0,"45,921M","221,113B","14,92"
2,OIBR3.SA,Oi S.A.,"1,8400","+0,0300","+1,66%",0,"146,878M","11,04B",N/A
3,MGLU3.SA,Magazine Luiza S.A.,"22,29","+0,01","+0,04%",0,"34,87M","144,078B","371,50"
4,VVAR3.SA,Via Varejo S.A.,"12,16","+0,55","+4,74%",0,"47,316M","19,42B","18,20"
5,FNAM11.SA,Fundo de Investimento Amazônia - FINAM,"0,1700","0,0000","0,00%",0,N/A,N/A,N/A
6,ITUB4.SA,Itaú Unibanco Holding S.A.,"28,21","+0,06","+0,21%",0,"39,085M","259,929B","14,63"
7,COGN3.SA,Cogna Educação S.A.,"4,1600","+0,1900","+4,79%",0,"50,068M","7,775B",N/A
8,ITSA4.SA,Itaúsa - Investimentos Itaú SA,"10,44","-0,03","-0,29%",0,"31,418M","90,063B","12,44"
9,IRBR3.SA,IRB-Brasil Resseguros S.A.,"6,18","+0,01","+0,16%",0,"44,676M","7,758B",N/A


Table with the sites

In [10]:
pd.DataFrame(stock_history).head(5)

,0
0,https://br.financas.yahoo.com/quote/PETR4.SA/h...
1,https://br.financas.yahoo.com/quote/BBDC4.SA/h...
2,https://br.financas.yahoo.com/quote/OIBR3.SA/h...
3,https://br.financas.yahoo.com/quote/MGLU3.SA/h...
4,https://br.financas.yahoo.com/quote/VVAR3.SA/h...


Historical stock price 

## Saving historical data

In [11]:
parent_dir = "C:/Users/carva/OneDrive/Documentos/Projects/Scraping"
os.chdir(parent_dir)
SAVE_FOLDER = "stock_price_history"
if not os.path.exists(SAVE_FOLDER): 
    os.mkdir(SAVE_FOLDER)
path = os.path.join(parent_dir, SAVE_FOLDER)
os.chdir(path)

In [11]:
test = []
for stock in stock_history:
    response = urlopen(stock)
    head = response.read()
    soup = BeautifulSoup(head, features="html.parser")
    pre_table = soup.find('tbody').find_all('tr')
    for all_rows in pre_table:
        col = all_rows.find_all('td')
        if not col[1].has_attr("colspan"):
            test_aux = {'Data_ref': col[0].get_text(), 'Abrir': col[1].get_text(), 'Alta': col[2].get_text(),
                    'Baixa': col[3].get_text(), 'Fechamento': col[4].get_text(), 'Fechamento_aj': col[5].get_text(),
                    'Volume' : col[6].get_text()} 
            test.append(test_aux)
        else:
            continue
    data_stock_hist = pd.DataFrame(test)
    name_csv = stock.split("/")[4].removesuffix(".SA") + ".csv"
    data_stock_hist.to_csv(name_csv)

Authomatization search of stock information

In [114]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 

In [116]:
stocks = ['OIBR3', 'PETR4']
for stock in stocks:
    driver = webdriver.Chrome(executable_path = 'C:\\Program Files\\Chromedriver\\chromedriver.exe')
    driver.get("https://www.google.com/")
    elem = driver.find_element_by_name("q")
    elem.clear()
    elem.send_keys(stock)
    elem.send_keys(Keys.RETURN)
    elem_new = driver.find_element_by_class_name("MUFPAc")
    notic = elem_new.find_elements_by_tag_name("a")
    contador = [i for i in range(len(notic))]
    for n in contador:
        if notic[n].text == "Notícias":
            notic[n].click()
            break
        else:
            continue
    time.sleep(5)        
    driver.quit()